In [4]:
import pandas as pd
import numpy as np
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction
from chromadb.config import Settings
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from langdetect import detect
from gensim.models import Word2Vec
import spacy
import nltk
import re
import chromadb
import string
import uuid

In [5]:
french_stopwords = set(stopwords.words('french'))

In [8]:
article = pd.read_csv("fstt-articles.csv")
clubs = pd.read_csv("fstt-clubs-info.csv")
dep = pd.read_csv("fstt-departements-info.csv")
formation = pd.read_csv("fstt-formation-initial.csv")

In [9]:
article.head()

,No,post_title,publish_date,post_content,post_link
0,1,CYBERSEC360° : PERSPECTIVES EN CYBERSÉCURITÉ A...,"mars 26, 2024",Un séminaire sera organisé le le samedi 30 Mar...,https://fstt.ac.ma/Portail2023/cybersec360-per...
1,2,PRÉSENTATION CONCOURS D’INNOVATION TDC,"mars 26, 2024",À la suite de l’appel à candidature au concour...,https://fstt.ac.ma/Portail2023/presentation-co...
2,3,SÉMINAIRE : WATER RESOURCES MANAGEMENT FOR SUS...,"mars 26, 2024","À l’occasion de la Journée mondiale de l’eau, ...",https://fstt.ac.ma/Portail2023/seminaire-water...
3,4,ORACLE MOROCCO R&D CENTER : PROGRAMME DE STAGE...,"mars 23, 2024",Oracle Morocco R&D Center lance l’appel aux ca...,https://fstt.ac.ma/Portail2023/oracle-morocco-...
4,5,APPEL À PROJET : «TERRITORY DEVELOPMENT CHALLE...,"mars 22, 2024",Le Centre régional d’investissement (CRI-TTA) ...,https://fstt.ac.ma/Portail2023/appel-a-projet-...


In [10]:
clubs.head()

,No,club_name,departement_info,club_link
0,1,GREENOLOGY,Le Club de l’environnement de la FST de Tanger...,https://fstt.ac.ma/Portail2023/greenology/
1,2,CADAC,https://www.facebook.com/CADAC.FSTT,https://fstt.ac.ma/Portail2023/club-d-art-dram...
2,3,CLUB GÉNIE CIVIL,Le Club Génie civil est un club scientifique d...,https://fstt.ac.ma/Portail2023/club-genie-civil/
3,4,CLUB LES SOPHISTES,https://www.facebook.com/SophistesFSTT,https://fstt.ac.ma/Portail2023/club-les-sophis...
4,5,Design & Photography,"Le « Club Design & Photography », est une fili...",https://fstt.ac.ma/Portail2023/club-design-pho...


In [11]:
dep.head()

,No,departement_name,departement_info
0,1,GÉNIE INFORMATIQUE,Chef : Pr.EL BRAK Mohamed \nEmail : melbr...
1,2,GÉNIE CHIMIQUE,Chef : Pr.CHABBI Mohamed\nEmail : mchabbi@uae....
2,3,SCIENCES DE LA TERRE,Chef : Pr.BOULAASSAL Hakim\nEmail : h.boulaass...
3,4,GÉNIE MÉCANIQUE,Chef : Pr.ELAYACHI Ilham\n \nEmail : ...
4,5,SCIENCES DE LA VIE,Chef : Pr.HASSANI ZERROUK Mounir\nEmail : mhas...


In [12]:
formation.head()

,No,mst_name,formation_type,mst_link,mst_objectif,mst_program,mst_skills,mst_Coord
0,1,MST : Analyse Appliquée et Ingénierie Statistique,mst,https://fstt.ac.ma/Portail2023/mst-analyse-app...,Le Master Analyse Appliquée et Ingénierie Stat...,"{'Semestre 1': ['', ' Statistique mathématique...",Donner une formation mathématique et statistiq...,Coordinnateur pédagogique : Pr.EL HALIMI Rach...
1,2,MST : Bases Cellulaires et Moléculaires en Bio...,mst,https://fstt.ac.ma/Portail2023/mst-bases-cellu...,Le master BCMB vise la formation des spécialis...,"{'Semestre 1': ['', ' Techniques Expérimentale...","Secteur public : chercheur, responsable, ingén...",Coordinnateur pédagogique : Pr.Mohamed NHIRI ...
2,3,"MST : Environnement, Aquaculture et Développem...",mst,https://fstt.ac.ma/Portail2023/mst-environneme...,"Encore peu développée au Maroc, l’aquaculture ...","{'Semestre 1': ['', ' Écosystèmes aquatiques e...",NaN,Coordinnateur pédagogique : Pr.Hassan Er-RAIOU...
3,4,MST : Géoressources Energétiques et Réservoirs,mst,https://fstt.ac.ma/Portail2023/mst-georessourc...,L’activité accrue dans la recherche en géoscie...,"{'Semestre 1': ['', ' Géodynamique et Analyse ...",NaN,Coordinnateur pédagogique : Pr.Mohamed Najib Z...
4,5,MST : Génie Civil,mst,https://fstt.ac.ma/Portail2023/mst-genie-civil/,Le programme vise la formation des cadres en G...,"{'Semestre 1': ['', ' Mathématiques Pour l’Ing...",NaN,Coordinnateur pédagogique : Pr.Dkiouak Rachid ...


In [6]:
def is_french(text):
    try:
        return detect(text) == 'fr'
    except:
        return False
def preprocessing(text):
    # Convert to string and lowercase
    text = str(text).lower()
    
    # Remove URLs
    # text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    
    # Remove specific unwanted characters
    text = re.sub(r'«|»|“|”|’|‘', '', text)
    
        # Tokenization
    """tokens = sent_tokenize(text, language="french")
    print("Tokenization :" ,tokens)"""
    # Check if text is in French
    if not is_french(text):
        return ''
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # # Tokenization using RegexpTokenizer
    # tokenizer = RegexpTokenizer(r'\w+')
    # tokens = tokenizer.tokenize(text)
    
    tokens = word_tokenize(text, language="french")
    tokens = [word for word in tokens if word not in french_stopwords]
    
    
    # Stemming
    stemmer =nltk.stem.snowball.FrenchStemmer()
    tokens_stemmed = [stemmer.stem(word) for word in tokens]
    
    # Join tokens back into a single string
    return ' '.join(tokens_stemmed)

In [6]:
article.drop(columns=["No"], inplace=True)
clubs.drop(columns=["No"], inplace=True)
formation.drop(columns=["No"], inplace=True)

In [7]:
article["post_title"] = article["post_title"].apply(preprocessing)
article["post_content"] = article["post_content"].apply(preprocessing)
clubs["departement_info"] = clubs["departement_info"].apply(preprocessing)
formation["mst_name"] = formation["mst_name"].apply(preprocessing)
formation["mst_objectif"] = formation["mst_objectif"].apply(preprocessing)
formation["mst_program"] = formation["mst_program"].apply(preprocessing)
formation["mst_skills"] = formation["mst_skills"].apply(preprocessing)
formation["mst_Coord"] = formation["mst_Coord"].apply(preprocessing)

In [8]:
# create EF with custom endpoint
ef = OllamaEmbeddingFunction(
    model_name="nomic-embed-text",
    url="http://localhost:11434/api/embeddings",
)

In [9]:
# Configure the ChromaDB client with persistence
persist_directory = "/home/idriss/Desktop/chroma_db"
client = chromadb.PersistentClient(path=persist_directory)

# client = chromadb.Client(Settings(persist_directory=persist_directory))

In [11]:
# client.delete_collection(name="text_embeddings")

In [10]:
# Create a collection in ChromaDB with the OllamaEmbeddingFunction
collection = client.create_collection(name="text_embeddings")

In [11]:
import uuid

def process_and_store_embeddings(dataframe, column_names, collection):
    ids = []
    metadatas = []
    documents = []

    for idx, row in dataframe.iterrows():
        doc_metadata = {}
        for key, value in row.items():
            if key not in column_names:
                doc_metadata[key] = value  # Leave non-string fields as is
        
        for column in column_names:
            sentence = row[column]
            if sentence is not None and sentence != '':
                ids.append(str(uuid.uuid1()))
                metadatas.append(doc_metadata)
                documents.append(sentence)
    collection.add(
        ids=ids,
        #embeddings=embeddings,
        metadatas=metadatas,
        documents=documents
    )

# Assuming article, clubs, formation DataFrames are already preprocessed
# Store embeddings, metadata, and documents from each DataFrame
process_and_store_embeddings(article, ["post_title", "post_content"], collection)
process_and_store_embeddings(clubs, ["departement_info"], collection)
process_and_store_embeddings(formation, ["mst_name", "mst_objectif", "mst_program", "mst_skills", "mst_Coord"], collection)

print("Embeddings, metadata, and documents stored in ChromaDB successfully.")


Embeddings, metadata, and documents stored in ChromaDB successfully.


In [12]:
collection.peek()

{'ids': ['6d206776-1d3b-11ef-aa8c-d46d6d576da4',
  '6d206777-1d3b-11ef-aa8c-d46d6d576da4',
  '6d206778-1d3b-11ef-aa8c-d46d6d576da4',
  '6d206779-1d3b-11ef-aa8c-d46d6d576da4',
  '6d20677a-1d3b-11ef-aa8c-d46d6d576da4',
  '6d20677b-1d3b-11ef-aa8c-d46d6d576da4',
  '6d20677c-1d3b-11ef-aa8c-d46d6d576da4',
  '6d20677d-1d3b-11ef-aa8c-d46d6d576da4',
  '6d20677e-1d3b-11ef-aa8c-d46d6d576da4',
  '6d20677f-1d3b-11ef-aa8c-d46d6d576da4'],
 'embeddings': [[-0.05194784328341484,
   -0.02235429920256138,
   -0.07353021949529648,
   -0.09421021491289139,
   0.026084665209054947,
   0.031177420169115067,
   0.036494456231594086,
   -0.013198098167777061,
   -0.0064868382178246975,
   0.004847310483455658,
   -0.0019958948250859976,
   -0.027504069730639458,
   -0.05993557348847389,
   0.02888396754860878,
   -0.02331400103867054,
   -0.004047835245728493,
   -0.012942984700202942,
   -0.02473314106464386,
   0.051636792719364166,
   -0.030879177153110504,
   -0.016608642414212227,
   -0.049243416637182236

In [22]:
# Load the existing collection
collection = client.get_collection(name="text_embeddings")

# Perform a query
query = "informatique"
results = collection.query(
    query_texts=[query],
    n_results=10
)

for result in results['documents']:
    print(result)


['gen informat', 'avoir cultur bas scientif acquer bas solid axe majeur fondamental disciplin informat algorithm programm bas don technolog web system dexploit réseau informat développ savoirfair techniqu informat technolog objet informat distribu architectur clientserveur ntier appliqu hétérogen', 'mathémat informat décisionnel', 'ingénier développ dappliqu informat', 'licenc scienc techniqu permet ingénier développ dappliqu informat étudi davoir cultur bas scientifiquedacquer bas solid axe majeur fondamental disciplin informat algorithm programm bas don system dexploit réseauxd développ savoirfair techniqu informat technolog objet informat distribu architectur clientserveur ntier appliqu hétérogènesdamélior niveau danglaisdavoir cultur dentrepris suffis fair idé concret mond travail', 'permettr étudi dacquer connaiss mathémat fondamental appliqu auss scienc don informatiqueapport étudi format solid statist scienc don tout permet dacquer cultur scientif général favoris integr futurdév